# Playground CHATGPT
Author : Patricio Ortiz

The main idea of this notebook is to handle the data of some experiments, and be able to work with them in order to get a CHATGPT response with the diagnosis and the actions proposed to the client.

## Libraries

In [1]:
import pandas as pd
import numpy as np
import openai
import os

from aux_functions import *

openai.api_key = os.getenv("OPENAI_API_KEY")

## Data Handle

In [3]:
# Data samples -> Main Table
df_sam = pd.read_csv('../Data/samples_v2.csv', low_memory=False).iloc[:,:-5].drop(columns=[' id_protocol', ' component_continuity', ' lubricant_continuity', ' machine_continuity', ' number_of_refills',' id_component_change_motive', ' id_lubricant_change_refill',' id_lubricant_change_refill_motive', ' lubricant_change_refill_liters',' lubricant_change_refill_hours', ' state', ' suggestion']).replace("\\N", np.nan).dropna()
df_sam.loc[:,['id_sample', ' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client', ' id_anomaly_type',' id_tracking_state', ' id_condition']] = df_sam.loc[:,['id_sample', ' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client', ' id_anomaly_type',' id_tracking_state', ' id_condition']].astype(int)

# Data components -> Mechanic Components
df_com = pd.read_csv('../Data/components.csv')
df_com.loc[:,['id_component', 'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client']] = df_com.loc[:,['id_component', 'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client']].astype(int)

# Datos ensayos -> Chemical components
df_qui = pd.read_csv('../Data/essays.csv')

# Datos resultados -> Values of meditions and limits
# df_res = pd.read_csv('../Data/results.csv', low_memory=False).replace("\\N", np.nan)
# df_res = df_res[~df_res.id_essay.isin([47,48,57])]
# df_res['value'] = df_res['value'].str.rstrip('.')
# df_res['value'] = df_res['value'].str.replace('\.\.', '.', regex=True)
# df_res['value'] = df_res['value'].str.replace('^(\.)+', '', regex=True)
# df_res.dropna(subset=['lsm','lsc','lim','lic'], how='all', inplace=True)
# df_res.to_csv('../Data/results_v2.csv', index=False)
df_res = pd.read_csv('../Data/results_v2.csv', low_memory=False)
df_res.loc[:,['lic','lim','lsm','lsc']] = df_res.loc[:,['lic','lim','lsm','lsc']].astype(float)

# Datos condiciones -> Conditions label 
df_con = pd.read_csv('../Data/conditions.csv')

# Datos tracking -> Tracking state label
df_tra = pd.read_csv('../Data/tracking.csv')

# Datos anomalias -> Anomalies label
df_ano = pd.read_csv('../Data/anomalies.csv')

In [4]:
# Recover labels
dfk = pd.merge(df_sam, 
               df_con,
               left_on=' id_condition',
               right_on='id').drop(columns=['id', ' id_condition']).rename(columns={' name':' condition'})    

dfk = pd.merge(dfk, 
               df_tra,
               left_on=' id_tracking_state',
               right_on='id').drop(columns=['id', ' id_tracking_state']).rename(columns={' name':' tracking_state'})   
 
dfk = pd.merge(dfk, 
               df_ano,
               left_on=' id_anomaly_type',
               right_on='id').drop(columns=['id', ' id_anomaly_type']).rename(columns={' name':' anomaly'})    


In [5]:
list_cases = ['<1', 'NSD', '9.8.05', '0.000.00', '0.0.00', '0.0.0', '0.004.05', '0-00', 
              '0.2.25', '89.88.56', '<0.1', 'N', 'N/', '/NA', '1.2.446', '0.0.2235', '10.0.37',
              '9.69.6', 'nn', 'n', '', '144.580.0', '2.0.326514', '14.20.154111', '9.9.56',
              '4.19.00', '9.9.2' ]

df_aux = df_res[df_res['value'].isin(list_cases)]
ids = df_aux['id_sample'].unique()
df_res = df_res[~df_res['id_sample'].isin(ids)]
df_res.loc[:,'value'] = df_res.loc[:,'value'].astype(float).round(1)

# Get dataframe with values
df_val = pd.merge(df_res, df_qui, 
                  left_on='id_essay',
                  right_on='id').drop(columns=['id','id_essay']).rename(columns={'value':'valor',
                                                                                 'name':'elemento',})

df2 = df_val.copy()

df_val.rename(columns={ 'lsm' :'limite superior marginal',
                        'lsc' :'limite superior condenatorio',
                        'lim' :'limite inferior marginal',
                        'lic' :'limite inferior condenatorio', }, inplace=True)

df_val = df_val[(df_val.valor >= df_val['limite superior marginal']) | (df_val.valor <= df_val['limite inferior marginal'])]
df_val = df_val[df_val.valor > 0]

df_val.tail()

,id_sample,valor,limite inferior condenatorio,limite inferior marginal,limite superior marginal,limite superior condenatorio,elemento
6821302,384807,3.5,NaN,NaN,1.5,3.0,Hollín %
6821394,385155,1.5,NaN,NaN,1.5,3.0,Hollín %
6821423,385292,2.0,NaN,NaN,1.5,3.0,Hollín %
6821451,385368,2.6,NaN,NaN,1.5,3.0,Hollín %
6821453,385374,1.7,NaN,NaN,1.5,3.0,Hollín %


In [9]:
# Obtain data of components
dfi = pd.merge(dfk, df_com,
               left_on=[' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client'],
               right_on=['id_component', 'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client']).drop(columns=['id_component',' id_component', ' id_component_type', ' id_machine', ' id_machine_type', ' id_site', ' id_client',
                                                                                                                                      'id_component_type', 'id_machine', 'id_machine_type', 'id_site', 'id_client', 'id_component_brand', 'id_component_model',
                                                                                                                                      'id_machine_brand', 'id_machine_model'])

dfi.head() # THIS IS USED AS DF_GENERAL

,id_sample,condition,tracking_state,anomaly,name_component,name_component_brand,name_component_model,name_component_type,name_machine,name_machine_type,name_machine_brand,name_machine_model,name_site,name_client
0,54065,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-1,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
1,57247,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-1,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
2,59059,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-1,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
3,54066,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-2,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM
4,57246,Normal,Normal,Normal,REDUCTOR,NaN,NaN,REDUCTOR,CT-2,CORREA TRANSPORTADORA,NaN,NaN,MOP-H,SQM


In [79]:
# Function to determine the limit surpassed and its value
def find_limit_surpassed(row):
    if row['valor'] <= row['limite inferior condenatorio']:
        return 'limite inferior condenatorio', row['limite inferior condenatorio']
    elif row['valor'] >= row['limite superior condenatorio']:
        return 'limite superior condenatorio', row['limite superior condenatorio']
    elif row['valor'] <= row['limite inferior marginal']:
        return 'limite inferior marginal', row['limite inferior marginal']
    elif row['valor'] >= row['limite superior marginal']:
        return 'limite superior marginal', row['limite superior marginal']
    else:
        return 'No limit surpassed', None
    
def value_prompt()

# # Apply the function to create new columns indicating the limit surpassed and its value
# df_val[['limit_surpassed', 'limit_surpassed_value']] = df_val.apply(find_limit_surpassed, axis=1, result_type='expand')

# # Select the desired columns in the final DataFrame
# final_df = df_val[['elemento', 'valor', 'limit_surpassed', 'limit_surpassed_value']]


In [68]:
val = df_val[df_val.id_sample == id]
val

,id_sample,valor,limite inferior condenatorio,limite inferior marginal,limite superior marginal,limite superior condenatorio,elemento,limit_surpassed,limit_surpassed_value
92498,170166,31.1,NaN,NaN,30.0,40.0,Fierro,limite superior marginal,30.0
993402,170166,267.7,NaN,NaN,5.0,15.0,Cobre,limite superior marginal,5.0
1581609,170166,2.9,NaN,NaN,1.5,3.0,Estaño,limite superior marginal,1.5
2150816,170166,30.7,NaN,NaN,14.0,17.0,Silicio,limite superior marginal,14.0
3079811,170166,20.0,NaN,NaN,20.0,40.0,Índice PQ,No limit surpassed,NaN
6043485,170166,10.7,11.0,12.0,16.1,17.1,Viscosidad cinemática @ 100°C,limite inferior condenatorio,11.0
6730094,170166,0.8,NaN,NaN,0.3,0.5,Dilución por combustible,limite superior marginal,0.3


In [87]:
val[['limite transgredido', 'valor limite']] = val.apply(find_limit_surpassed, axis=1, result_type='expand')
val

C:\Users\Visco\AppData\Local\Temp\ipykernel_12280\2214600715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[['limite transgredido', 'valor limite']] = val.apply(find_limit_surpassed, axis=1, result_type='expand')
C:\Users\Visco\AppData\Local\Temp\ipykernel_12280\2214600715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val[['limite transgredido', 'valor limite']] = val.apply(find_limit_surpassed, axis=1, result_type='expand')


,id_sample,valor,limite inferior condenatorio,limite inferior marginal,limite superior marginal,limite superior condenatorio,elemento,limit_surpassed,limit_surpassed_value,limite transgredido,valor limite
92498,170166,31.1,NaN,NaN,30.0,40.0,Fierro,limite superior marginal,30.0,limite superior marginal,30.0
993402,170166,267.7,NaN,NaN,5.0,15.0,Cobre,limite superior condenatorio,15.0,limite superior condenatorio,15.0
1581609,170166,2.9,NaN,NaN,1.5,3.0,Estaño,limite superior marginal,1.5,limite superior marginal,1.5
2150816,170166,30.7,NaN,NaN,14.0,17.0,Silicio,limite superior condenatorio,17.0,limite superior condenatorio,17.0
3079811,170166,20.0,NaN,NaN,20.0,40.0,Índice PQ,limite superior marginal,20.0,limite superior marginal,20.0
6043485,170166,10.7,11.0,12.0,16.1,17.1,Viscosidad cinemática @ 100°C,limite inferior condenatorio,11.0,limite inferior condenatorio,11.0
6730094,170166,0.8,NaN,NaN,0.3,0.5,Dilución por combustible,limite superior condenatorio,0.5,limite superior condenatorio,0.5


In [88]:
final_df = val[['elemento', 'valor', 'limite transgredido', 'valor limite']]
final_df

,elemento,valor,limite transgredido,valor limite
92498,Fierro,31.1,limite superior marginal,30.0
993402,Cobre,267.7,limite superior condenatorio,15.0
1581609,Estaño,2.9,limite superior marginal,1.5
2150816,Silicio,30.7,limite superior condenatorio,17.0
3079811,Índice PQ,20.0,limite superior marginal,20.0
6043485,Viscosidad cinemática @ 100°C,10.7,limite inferior condenatorio,11.0
6730094,Dilución por combustible,0.8,limite superior condenatorio,0.5


In [74]:
comp = component_prompt(dfi[dfi.id_sample == id][['name_component','name_component_brand','name_component_model','name_component_type','name_machine','name_machine_type','name_machine_brand','name_machine_model']])

In [75]:
comp

'Componente: Motor Diesel\nMáquina: Camión Tolva de 11 m3 - 03-105610'

In [90]:
p = 'Analiza una muestra para el siguiente equipo:\n' + comp + '\n' + 'Los valores de la muestra son:\n' + final_df.to_string(index=False)

In [91]:
print(p)

Analiza una muestra para el siguiente equipo:
Componente: Motor Diesel
Máquina: Camión Tolva de 11 m3 - 03-105610
Los valores de la muestra son:
                     elemento  valor          limite transgredido  valor limite
                       Fierro   31.1     limite superior marginal          30.0
                        Cobre  267.7 limite superior condenatorio          15.0
                       Estaño    2.9     limite superior marginal           1.5
                      Silicio   30.7 limite superior condenatorio          17.0
                    Índice PQ   20.0     limite superior marginal          20.0
Viscosidad cinemática @ 100°C   10.7 limite inferior condenatorio          11.0
     Dilución por combustible    0.8 limite superior condenatorio           0.5


In [92]:
context = {"role": "system",
                "content": "Eres un ingeniero mecanico, especialista en equipos mineros y debes realizar diagnosticos precisos sobre las medidas de un equipo, entregando comentarios breves respecto a los análisis de aceite realizados y recomendaciones concretas de mantención. Tus respuestas deben ser de 150 palabras o menos"}
messages = [context]


# Add user message containing the prompt
messages.append({"role": "user", "content": p})

# Call OpenAI API to generate the response based on the conversation history
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)

response.choices[0].message.content

SyntaxError: 'return' outside function (719360099.py, line 15)

In [94]:
print(response.choices[0].message.content)

El análisis de aceite del motor diesel del Camión Tolva de 11 m3 - 03-105610 indica que hay varias preocupaciones. El alto contenido de cobre y silicio es una señal de un desgaste excesivo en las piezas del motor. El alto contenido de hierro también indica una probable disminución de la vida útil del motor. Además, se ha detectado una bajo viscosidad cinemática @ 100°C, lo que puede causar una lubricación deficiente del motor y aumentar el desgaste de las piezas. 

Se recomienda un análisis más profundo del motor para determinar qué componentes necesitan ser reparados o reemplazados. Se recomienda realizar una limpieza completa del sistema de aceite y sustituir el aceite por uno de alta calidad que cumpla con las especificaciones del fabricante. Además, se aconseja realizar cambios de aceite y filtros de manera más frecuente para controlar el desgaste en el motor y reducir los niveles de contaminación.  También es recomendable controlar la dilución por combustible en el futuro para evi

## Functions

In [71]:
# Function to generate the prompt to chatgpt
def generate_prompt(gen, val):
    """
    Generates the main prompt for comments based on GPT-3.

    Args:
        row (pd.Series): Series representing a row from the table, containing the component information.
        val (pd.DataFrame): DataFrame containing the values of the sample.

    Returns:
        str: Generated prompt for GPT-3 chat interaction.
    """
    # Execute functions to get characteristics of sample
    com_prompt = component_prompt(gen)  # Call component_prompt() function to generate the component prompt for the sample
    val_prompt = val.to_string(index=False)  # Convert the DataFrame 'val' to a string representation

    # Unify prompts
    prompt = 'Analiza una muestra para el siguiente equipo:\n' + com_prompt + '\n' + 'Los valores de la muestra son:\n' + val_prompt

    return prompt

def component_prompt(row):
    """
    Generates the prompt that extract the info from component table using GPT-3.

    Args:
        row (pd.Series): Series representing a row from the table.

    Returns:
        str: Generated component prompt for GPT-3 interaction.
    """

    # Add context for the interaction with GPT-3
    context = {"role": "system",
                "content": "Eres un asistente que resume la información de una tabla a breves bullet points, evitando la redundancia"}
    messages = [context]

    # Set previous interaction to refine results
    ## Example 1
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            'name_component name_component_brand name_component_model name_component_type name_machine name_machine_type name_machine_brand name_machine_model\n  MOTOR DIESEL          CATERPILLAR                3516B        MOTOR DIESEL      CAEX 53      CAMIONES 793        CATERPILLAR               793B'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Motor Diesel Caterpillar 3516B\nMáquina: Camión Caterpillar 793B"})

    ## Example 2
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            'name_component name_component_brand name_component_model name_component_type name_machine name_machine_type name_machine_brand name_machine_model\n  GRASA AC-100                  NaN                  NaN            DEPOSITO CAMION GRASA            CAMION                NaN                NaN'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Deposito de grasa\nMáquina: Camión grasa"})

    ## Example 3
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            'name_component name_component_brand name_component_model name_component_type        name_machine name_machine_type name_machine_brand name_machine_model\n      REDUCTOR          NEU-SOCOFER                  NaN            REDUCTOR ASPIRADOS DE VIA 01  Aspirado de Vias        NEU-SOCOFER                NaN'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Reductor Neu-socofer\nMáquina: Aspirados de via Neu-socofer"})

    ## Example 4
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            'name_component name_component_brand name_component_model name_component_type name_machine     name_machine_type name_machine_brand name_machine_model\n      REDUCTOR                  NaN                  NaN            REDUCTOR         CT-2 CORREA TRANSPORTADORA                NaN                NaN'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Reductor\nMáquina: Correa Transportadora"})

    ## Example 5
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            'name_component name_component_brand name_component_model name_component_type name_machine      name_machine_type name_machine_brand name_machine_model\n  MOTOR DIESEL              CUMMINS                  NaN        MOTOR DIESEL     CAEX-726 CAMIONES DE EXTRACCIÓN            KOMATSU              730-E'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Motor Diesel Cummins\nMáquina: Camión Komatsu 730-E"})
    
    ## Example 6
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            '    name_component name_component_brand name_component_model name_component_type name_machine name_machine_type name_machine_brand name_machine_model\nACEITE MOTOR 15W40                  NaN                  NaN        MOTOR DIESEL  DESCONOCIDO       DESCONOCIDO                NaN                NaN'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Aceite motor Diesel 15W40\n"})

    ## Example 7
    messages.append({"role": "user", "content": """Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            'name_component name_component_brand name_component_model name_component_type                   name_machine name_machine_type name_machine_brand name_machine_model\n  MOTOR DIESEL                  NaN                  NaN        MOTOR DIESEL CAMION TOLVA 11 M3 - 03-105610    CAMIONES TOLVA                NaN                NaN'
                                            """})
    messages.append({"role": "assistant", "content": "Componente: Motor Diesel\nMáquina: Camión Tolva"})

    # Add user message providing the specific row information
    messages.append({"role": "user", "content": f"""Analiza los componentes y características de una máquina identificada en la siguiente tabla:
                                            {row.to_string(index=False)}
                                            """})

    # Call OpenAI API to generate the response based on the conversation history
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content  # Return the content of the assistant's response



In [58]:
r = """Se detecta concentración de metales de desgaste por Fierro 31,09 ppm y Cobre 267,72 ppm, evidenciando posible abrasión excesiva en cojinetes y bujes de turbo. Silicio 30,72 ppm señala ingesta excesiva de polvo ambiental. Análisis fisicoquímico detecta material ferroso en suspensión Índice PQ 20,0. Se evidencia Dilución por combustible de 0,8%, detectando posible anormalidad en bomba de transferencia o sellos de inyectores, provocando una baja ligera en viscosidad en lubricante.
Se recomienda priorizar cambio de lubricante y elementos filtrantes, evaluar presiones en sistema de lubricación y saturación temprana de filtros, mantener seguimiento riguroso cada 50 hrs.
"""

In [59]:
r

'Se detecta concentración de metales de desgaste por Fierro 31,09 ppm y Cobre 267,72 ppm, evidenciando posible abrasión excesiva en cojinetes y bujes de turbo. Silicio 30,72 ppm señala ingesta excesiva de polvo ambiental. Análisis fisicoquímico detecta material ferroso en suspensión Índice PQ 20,0. Se evidencia Dilución por combustible de 0,8%, detectando posible anormalidad en bomba de transferencia o sellos de inyectores, provocando una baja ligera en viscosidad en lubricante.\nSe recomienda priorizar cambio de lubricante y elementos filtrantes, evaluar presiones en sistema de lubricación y saturación temprana de filtros, mantener seguimiento riguroso cada 50 hrs.\n'

In [ ]:
def excecute_prompt(prompt):
    """
    Executes a chat prompt with GPT-3 and retrieves the response.

    Args:
        prompt (str): Prompt for GPT-3 chat interaction.

    Returns:
        str: Generated response from GPT-3.
    """

    # Add context for the interaction with GPT-3
    context = {"role": "system",
                "content": "Eres un ingeniero mecanico, especialista en equipos mineros y debes realizar diagnosticos precisos sobre las medidas de un equipo, entregando comentarios breves respecto a los análisis de aceite realizados y recomendaciones concretas de mantención. Tus respuestas deben ser de 150 palabras o menos"}
    messages = [context]

    # Set previous interaction to refine results
    ## Example 1
    messages.append({"role": "user", "content": 'Analiza una muestra para el siguiente equipo:\nComponente: Aceite motor Diesel 15W40\nLos valores de la muestra son:\n valor  limite inferior condenatorio  limite inferior marginal  limite superior marginal  limite superior condenatorio                     elemento\n   8.3                           NaN                       NaN                       0.1                           0.3            Contenido de agua\n 144.6                         102.0                     108.0                     132.0                         138.0 Viscosidad cinemática @ 40°C'})
    messages.append({"role": "assistant", "content": 'Se aprecian niveles de desgaste y contaminación externa entre límites permisibles, sin embargo, se detecta contenido de agua 8,3% en volumen de muestra. Grado de viscosidad sobre límite superior condenatorio 144,6 por posibles rellenos con lubricante de ISO VG mayor.\nSe sugiere cambio de lubricante y mantener seguimiento riguroso cada 125 horas, para evidenciar alzas abruptas de sodio y potasio por eventual traspaso de refrigerante. Monitorear consumos de aceite y gases blancos en salida de escape.'})

    

    # Add user message containing the prompt
    messages.append({"role": "user", "content": prompt})

    # Call OpenAI API to generate the response based on the conversation history
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content  # Return the content of the assistant's response

In [43]:
def generate_comment(id, df_general=dfi, df_values=df_val):
  """
  Generates a comment based on the sample id and its associated data.

  Args:
      id (int): The sample id for which to generate the comment.
      df_general (pandas.DataFrame, optional): The DataFrame containing general information about the samples.
          Defaults to dfi.
      df_values (pandas.DataFrame, optional): The DataFrame containing the values associated with the samples.
          Defaults to df_val.

  Returns:
      tuple: A tuple containing the generated response comment and the corresponding prompt.

  Raises:
      None

  Notes:
      - This function assumes that the 'df_general' DataFrame has the columns 'id_sample', 'name_component',
        'name_component_type', 'name_machine', and 'name_machine_type'.
      - This function assumes that the 'df_values' DataFrame has the columns 'id_sample' and any other necessary
        columns containing the values of interest.

  """

  # Get the sample rows
  ## General row
  gen = df_general[df_general.id_sample == id][['name_component','name_component_brand','name_component_model','name_component_type','name_machine','name_machine_type','name_machine_brand','name_machine_model']]
  ## Values rows
  val = df_val[df_val.id_sample == id].drop(columns=['id_sample'])

  # If no limit is surpassed
  if val.shape[0] == 0:
    response = """Niveles de desgaste junto a contaminación externa dentro de los límites permisibles para el servicio.
    Grado de viscocidad y degradación normal de lubricante.
    Continuar con monitoreo de lubricante y componenete según plan de mantenimiento.
    """
    prompt = ''

  #If any limit is surpassed
  else:
    prompt = generate_prompt(gen, val)
    # response = excecute_prompt(prompt)
    response = ''

  return (response, prompt)

In [44]:
l = []
ids = dfi.iloc[-10:,0].values.tolist()

for i in ids:
  c = generate_comment(i)
  t1 = c[0]
  t2 = c[1]
  t3 = df2[df2.id_sample == i]
  t = (t1, t2, t3)
  
  l.append(t)

In [57]:
k

'Analiza una muestra para el siguiente equipo:\nComponente: Motor Diesel \n  Máquina: Camión Tolva de 11m3 - 03-105610\nLos valores de la muestra son:\n valor  limite inferior condenatorio  limite inferior marginal  limite superior marginal  limite superior condenatorio                      elemento\n  31.1                           NaN                       NaN                      30.0                          40.0                        Fierro\n 267.7                           NaN                       NaN                       5.0                          15.0                         Cobre\n   2.9                           NaN                       NaN                       1.5                           3.0                        Estaño\n  30.7                           NaN                       NaN                      14.0                          17.0                       Silicio\n  20.0                           NaN                       NaN                      20.0          

In [56]:
i = 1


print(ids[i])
print('\n')
print(l[i][0])
print('\n')
k = l[i][1]
print(l[i][1])
print('\n')
display(l[i][2])

170166





Analiza una muestra para el siguiente equipo:
Componente: Motor Diesel 
  Máquina: Camión Tolva de 11m3 - 03-105610
Los valores de la muestra son:
 valor  limite inferior condenatorio  limite inferior marginal  limite superior marginal  limite superior condenatorio                      elemento
  31.1                           NaN                       NaN                      30.0                          40.0                        Fierro
 267.7                           NaN                       NaN                       5.0                          15.0                         Cobre
   2.9                           NaN                       NaN                       1.5                           3.0                        Estaño
  30.7                           NaN                       NaN                      14.0                          17.0                       Silicio
  20.0                           NaN                       NaN                      20.0        

,id_sample,valor,lic,lim,lsm,lsc,elemento
92498,170166,31.1,NaN,NaN,30.0,40.0,Fierro
394967,170166,0.3,NaN,NaN,3.0,6.0,Cromo
695077,170166,3.0,NaN,NaN,5.0,10.0,Aluminio
993402,170166,267.7,NaN,NaN,5.0,15.0,Cobre
1291376,170166,0.8,NaN,NaN,5.0,10.0,Plomo
1581609,170166,2.9,NaN,NaN,1.5,3.0,Estaño
1858191,170166,5.3,NaN,NaN,20.0,25.0,Sodio
2150816,170166,30.7,NaN,NaN,14.0,17.0,Silicio
2571524,170166,4.0,NaN,NaN,10.0,15.0,Potasio
3079811,170166,20.0,NaN,NaN,20.0,40.0,Índice PQ


In [14]:
c1, p1 = generate_comment(385549)
print(c1,"\n")
print(p1)

La presencia de silicio en la muestra de aceite indica la existencia de desgaste en las piezas metálicas del motor. Debido a que el valor se encuentra dentro de los límites superiores marginales, no hay necesidad de tomar medidas inmediatas, pero se recomienda monitorear y realizar un análisis en la próxima mantención preventiva. No obstante, si el valor continúa aumentando, es recomendable tomar medidas correctivas cuanto antes, como revisar el sistema de filtración de aceite y el estado de los filtros.

Por otro lado, el valor de boro también se encuentra en los límites superiores marginales. Esto indica la presencia de un aditivo de aceite, que puede ser provocado por la utilización de un aceite de mayor calidad. En este caso, no es necesario tomar medidas específicas y se recomienda continuar usando el mismo tipo de aceite de alta calidad. No obstante, es importante seguir monitoreando los valores de boro en futuras muestras de aceite para asegurar que no se está acumulando por enc

In [18]:
c2, p2 = generate_comment(326629)
print(c2,"\n")
print(p2)

El análisis de aceite del motor diesel de la Grúa 02 revela algunos valores que están fuera del rango normal. El valor de sulfatación está ligeramente elevado, lo que indica la presencia de ácido sulfúrico en el aceite, posiblemente debido a una combustión incompleta del combustible. La dilución por combustible también es ligeramente alta, lo que puede deberse a una fuga en los inyectores de combustible que están inyectando combustible en el aceite. La viscosidad cinemática a 100°C está por debajo del rango normal, lo que indica que el aceite ha perdido su espesor. Esto puede deberse a la oxidación del aceite o a una excesiva dilución por combustible. 

Se recomienda llevar a cabo un cambio de aceite y filtrar el aceite para eliminar las impurezas. También se recomienda realizar una reparación en los inyectores de combustible y asegurarse de que no estén permitiendo que el combustible se mezcle con el aceite. Además de estos pasos, es importante monitorear periódicamente la condición d

In [19]:
c3, p3 = generate_comment(317431)
print(c3,"\n")
print(p3)

Los resultados del análisis de aceite del motor diesel del Camión de Extracción CAEX-726 muestran que la viscosidad cinemática a 100°C está por debajo del límite inferior marginal, lo que indica que es necesario reemplazar el aceite actual y agregar uno con una viscosidad adecuada. Además, la dilución por combustible está por encima del límite superior marginal, lo que indica la presencia excesiva de combustible en el aceite y la necesidad de revisar el sistema de inyección de combustible y reparar cualquier fuga.

Basado en los resultados, se recomienda realizar un reemplazo inmediato del aceite del motor y una revisión completa del sistema de inyección de combustible. Además, se sugiere que se realice una inspección general del motor diesel y sus componentes para determinar si existen otros problemas. Se debería reemplazar cualquier componente que presente desgaste o daño. Por último, se recomienda mantener un programa riguroso de mantenimiento preventivo para garantizar el buen func